Imports

In [2]:
import torch
import os
import sys
from torchvision import transforms
from IPython.display import Image 
import progressbar


script_root_dir = '../'
if script_root_dir not in sys.path:
    sys.path.append(script_root_dir)
    
from train import dataLoader
from train.configParser import ConfigParser
from helpers.color_PCA import Color_PCA

Parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
experimentName="testTrainingScript4"
dataPath="/home/elhamod/HGNN/data"

applyHorizontalFlip = True
applyHorizontalFlipProbability = 0.1

applyTranslation=True
applyTranslationAmount = 0.1

applyColorPCA=True
colorPCAperturbationMagnitude = 0.1

numOfAugmentedVersions = 10

cuda = 0

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)

Load dataset

In [4]:
# Get the dataset
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experiment_params = next(iter(config_parser.getExperiments()))
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
dataset = datasetManager.getDataset()

# disable normalization before augmentation.
dataset.toggle_image_loading(augmentation=False, normalization=False)

# Create appropriate loaders
loaderPerImage = torch.utils.data.DataLoader(dataset, batch_size=1)
datasetSize = len(dataset)
loaderAllImages = torch.utils.data.DataLoader(dataset, batch_size=datasetSize)
print("number of images", datasetSize)

Creating dataset...
550 len
Creating dataset... Done.
number of images 550


Generate data

In [5]:
color_pca = Color_PCA(loaderAllImages, colorPCAperturbationMagnitude)

with progressbar.ProgressBar(max_value=datasetSize) as bar:
    k=1
    for batch in loaderPerImage:
        original = batch['image'].squeeze()
        fileName = batch['fileName'][0]
        prefix, ext = os.path.splitext(fileName)

        if torch.cuda.is_available():
            original = original.cpu()

        for j in range(numOfAugmentedVersions):
            image = transforms.ToPILImage()(original)

            if applyHorizontalFlip:
                image = transforms.RandomHorizontalFlip(p=applyHorizontalFlipProbability)(image)
    #             display(image)

            if applyTranslation:
                RGBmean = (int(0.485*255), int(0.456*255), int(0.406*255))
                image = transforms.RandomAffine(degrees=0, translate=(applyTranslationAmount,
                                                                      applyTranslationAmount),
                                               fillcolor=RGBmean)(image)
    #             display(image)

            if applyColorPCA:
                image = transforms.ToTensor()(image).unsqueeze(0)
                image = transforms.Lambda(color_pca.perturb_color)(image).type(torch.float32) 

                image = transforms.ToPILImage()(image.squeeze())
    #             display(image)


            savedFileName = os.path.join(dataset.csv_processor.get_image_full_path(), prefix+"_aug"+str(j)+ext)
            print(savedFileName, "saved!")
            image.save(savedFileName)
            
        bar.update(k)
        k = k + 1

    
    

  1% (7 of 550) |                        | Elapsed Time: 0:00:00 ETA:   0:00:10

Transfrom images...


100% (550 of 550) |######################| Elapsed Time: 0:00:07 Time:  0:00:07
N/A% (0 of 550) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug0.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug1.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug2.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug3.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug4.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug5.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug6.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug7.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug8.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_017196_aug9.jpg saved!


  0% (2 of 550) |                        | Elapsed Time: 0:00:01 ETA:   0:05:06

...

/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug0.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug1.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug2.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug3.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug4.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug5.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug6.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug7.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug8.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_58884_aug9.jpg saved!


 99% (549 of 550) |##################### | Elapsed Time: 0:10:32 ETA:   0:00:01

/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug0.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug1.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug2.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug3.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug4.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug5.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug6.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug7.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug8.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_53547_aug9.jpg saved!


100% (550 of 550) |######################| Elapsed Time: 0:10:33 Time:  0:10:33


/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug0.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug1.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug2.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug3.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug4.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug5.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug6.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug7.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug8.jpg saved!
/home/elhamod/HGNN/data/INHS_cropped/images/INHS_FISH_14454_aug9.jpg saved!
